## Сборка единого датасета и его подготовка к EDA
Поскольку исходно данные для задачи даны нам в виде набора отдельных CSV-файлов, мы должны связать их в один датасет и провести некоторую предварительную обработку. После этого полученные датасет мы загрузим в streamlit-приложений, где выполним EDA и отобразим результаты пользователю. 

Исходные данные представляют собой набор CSV-файлов. Описание данных в них преведено в задании ([ссылка на ноутбук](https://colab.research.google.com/drive/1t_fUN60Hh_S9wsNPlUggiUiG9JCBcEmk?usp=sharing&pli=1&authuser=2#scrollTo=I7jn6IYXx6M9)) 

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Загрузка исходных CSV-файлов и сбор единого датасета

In [112]:
"""Загрузим все имеющиеся csv-файлы и выведем количество строк, столбцов и дубликатов в них"""
clients = pd.read_csv('datasets/D_clients.csv')
targets = pd.read_csv('datasets/D_target.csv')
job = pd.read_csv('datasets/D_job.csv')
salary = pd.read_csv('datasets/D_salary.csv')
last_credit = pd.read_csv('datasets/D_last_credit.csv')

loan = pd.read_csv('datasets/D_loan.csv')
close_loan = pd.read_csv('datasets/D_close_loan.csv')

pens = pd.read_csv('datasets/ignore/D_pens.csv')
work = pd.read_csv('datasets/ignore/D_work.csv')


def name_of(variable):
    """возвращает получает имя переменной"""
    ns = globals()
    return [name for name in ns if ns[name] is variable][0]

datasets = (clients, targets, job, salary, last_credit, loan, close_loan, pens, work)
for i in range(len(datasets)): 
    print(f'Данные в датасете {name_of(datasets[i])}. Строк: {datasets[i].shape[0]}. Столбцов: {datasets[i].shape[0]}. Дубликатов: {datasets[i].duplicated().sum()}')

Данные в датасете clients. Строк: 16000. Столбцов: 16000. Дубликатов: 0
Данные в датасете targets. Строк: 15223. Столбцов: 15223. Дубликатов: 0
Данные в датасете job. Строк: 15223. Столбцов: 15223. Дубликатов: 0
Данные в датасете salary. Строк: 15523. Столбцов: 15523. Дубликатов: 300
Данные в датасете last_credit. Строк: 15223. Столбцов: 15223. Дубликатов: 0
Данные в датасете loan. Строк: 21126. Столбцов: 21126. Дубликатов: 0
Данные в датасете close_loan. Строк: 21126. Столбцов: 21126. Дубликатов: 0
Данные в датасете pens. Строк: 2. Столбцов: 2. Дубликатов: 0
Данные в датасете work. Строк: 3. Столбцов: 3. Дубликатов: 0


Учитывая полученные характеристики датасетов и их описание в задании, слияние в единый датасет будем проводить следующим образом.
1) в качестве исходной возьмем таблицу clients
2) с помощью left join справа добавим к ней таблицы targets, job, salara (предварительно удалим дубликаты) и last_credit. Left join нужен в связи с тем, что не для всех строк в clients есть строки в этих таблицах.
3) предварительно соединим loan и close_loan, получив информацию обо всех кредитах. Учитывая, что количество строк в такой объединенной таблице будет равно 21126 (пока это предположение, но далее мы это и увидим), у некоторых клиентов есть информация о более чем одном кредите. В связи с этим, сделаем группировку полученной таблицы по клиентам, вычислив для каждого количество кредитов и количество закрытых кредитов. И уже такую таблицу будем присоединять справа данным о клиентам.

Таблицы pens и work мы проигнорируем, т.к. информация о том, работает или не работает человек, и на пенсии он, или нет, уже есть в таблице clients. Кроме того, данные в pens и work противоречат описанию датасета в задании, а также рациональным соображениям, заключающимся в том, что работающих клиентов банков и клиентов не на пенсии должно быть больше.

In [113]:
"""удаляем дубликаты в salary"""
salary.drop_duplicates(keep='first', inplace=True)

In [114]:
"""соединяем clients c target, job, salary, last_credit"""
result = pd.merge(clients, targets, how='left', left_on='ID', right_on='ID_CLIENT').drop(columns=['ID_CLIENT'])
result = pd.merge(result, job, how='left', left_on='ID', right_on='ID_CLIENT').drop(columns=['ID_CLIENT'])
result = pd.merge(result, salary, how='left', left_on='ID', right_on='ID_CLIENT').drop(columns=['ID_CLIENT'])
result = pd.merge(result, last_credit, how='left', left_on='ID', right_on='ID_CLIENT').drop(columns=['ID_CLIENT'])

In [115]:
"""собираем таблицу с агрегированными данными о кредитах по клиентам и ее также присоединяем к clients"""
agg_loan_data = pd.merge(loan, close_loan, how='inner', on='ID_LOAN').groupby('ID_CLIENT', as_index=False).agg(LOAN_COUNT =  ('ID_LOAN', 'count'), CLOSED_LOANS = ('CLOSED_FL', 'sum'))
result = pd.merge(result, agg_loan_data, how='left', left_on='ID', right_on='ID_CLIENT').drop(columns=['ID_CLIENT'])

In [117]:
result.shape

(16000, 27)

В результате слияния таблице получились те же 16000 записей о клиентах, но уже расширенных дополнительными данными. Проведем небольшой анализ и предобработку датасета

### Обработка датасета

Удалим из датасета столбцы с идентификаторами, т.к. теперь они нам не нужны

In [119]:
result.drop(columns = ['ID', 'AGREEMENT_RK'], inplace=True)

Посмотрим есть ли пропуски таргете.

In [121]:
result['TARGET'].isna().sum()

777

Как можно было видеть и ранее, для 777 записей о клиентах нет информации об их участии в маркетинговой кампании и об отклике на предложения банка. С точки зрения обучения моделей для прогнозирования возможного отклика эти данные нам не нужны. Мы могли бы оставить их для целей анализа, заполнив пропуски каким-то значением, говорящим о том, что клиент не участвовал в кампании. Но кроме этой информации для таких клиентов нет и других дополнительных данных. А значит аналих других характеристик тоже будет усложнен. В связи с этим *удалим данные о клиентах* без таргета. 
  

In [123]:
result = result[~result['TARGET'].isna()]
result.shape

(15223, 25)

Теперь посмотрим на пропуски в других столбцах

In [136]:
nas = result.isna().sum()
nas[nas.values > 0]

GEN_INDUSTRY    1367
GEN_TITLE       1367
JOB_DIR         1367
WORK_TIME       1368
dtype: int64

Видим, что пропуски есть в столбцах, связанных с информацией о работе клиента. Вспомнив, что среди данных о клиентах есть еще и информация о том, работает ли клиент, проверим, не являются ли клиенты с пропусками в указанных столбцах теми, кто не работает? Посмотрим на количество таких нетрудоустроенных клиентов. 

In [172]:
unemployed = result[result['SOCSTATUS_WORK_FL'] == 0]
unemployed.shape

(1376, 26)

Видно, количество клиентов без работы и и количество клиентов с пропусками в данных о работе почти совпадает, но все же отличается. Рассчитаем флаг, трудоустроен клиент или нет, на основе наличия или отсутствия данных о его работе, после чего выведем всех клиентов, для которых имеющееся и вычисленное значения не совпадают   

In [176]:
result['SOCSTATUS_WORK_FL_CALCULATED'] = (
    (
    ~(result['GEN_INDUSTRY'].isna() & 
      result['GEN_TITLE'].isna() & 
      result['JOB_DIR'].isna() & 
      result['WORK_TIME'].isna())
    ).astype(int))
inconsistent = result[result['SOCSTATUS_WORK_FL_CALCULATED'] != result['SOCSTATUS_WORK_FL']]
inconsistent[['SOCSTATUS_WORK_FL', 'SOCSTATUS_WORK_FL_CALCULATED', 'GEN_INDUSTRY', 'GEN_TITLE', 'JOB_DIR', 'WORK_TIME']]

,SOCSTATUS_WORK_FL,SOCSTATUS_WORK_FL_CALCULATED,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME
563,0,1,Образование,Руководитель среднего звена,Участие в основ. деятельности,420.0
2959,0,1,Металлургия/Промышленность/Машиностроение,Рабочий,Участие в основ. деятельности,48.0
4532,0,1,Образование,Работник сферы услуг,Вспомогательный техперсонал,240.0
5195,1,0,NaN,NaN,NaN,NaN
5648,0,1,Сельское хозяйство,Служащий,Участие в основ. деятельности,120.0
6369,0,1,Другие сферы,Служащий,Участие в основ. деятельности,9.0
7304,0,1,Другие сферы,Специалист,Участие в основ. деятельности,36.0
7447,0,1,Строительство,Специалист,Участие в основ. деятельности,NaN
11644,0,1,Ресторанный бизнес/Общественное питание,Рабочий,Участие в основ. деятельности,36.0
11691,0,1,Металлургия/Промышленность/Машиностроение,Специалист,Участие в основ. деятельности,360.0


Видим, что для 10 клиентов информация о трудоустройстве имеется, но соответствующий флаг SOCSTATUS_WORK_FL равен 0, хотя должен быть равен 1. 
В то же время, для одного клиента он равен 1, а данных о его трудоустройстве у банка нет. 

Будем считать данные о трудоустройстве первичными и заменим значение флага SOCSTATUS_WORK_FL на вычисленное нами значение. После чего снова посмотрим на количество клиентов без работы.

In [177]:
result['SOCSTATUS_WORK_FL'] = result['SOCSTATUS_WORK_FL_CALCULATED']
result.drop(columns=['SOCSTATUS_WORK_FL_CALCULATED'], inplace=True)

unemployed = result[result['SOCSTATUS_WORK_FL'] == 0]
unemployed.shape

(1367, 25)

Теперь видим, что количество нетрудоустроенных клиентов равно количеству клиентов с пропусками во всех столюцах с данными о работе. На всякий случай проверим, что это те же клиенты. Для этого посчитаем количество пропусков в указанных полях для клиентов без работы.
 

In [178]:
nas = unemployed.isna().sum()
nas[nas.values > 0]

GEN_INDUSTRY    1367
GEN_TITLE       1367
JOB_DIR         1367
WORK_TIME       1367
dtype: int64

In [180]:
result

,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,...,GEN_TITLE,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,LOAN_COUNT,CLOSED_LOANS
0,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,...,Работник сферы услуг,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,5588.0,6.0,1000.0,1.0,0.0
1,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,...,Специалист,Участие в основ. деятельности,5.0,от 10000 до 20000 руб.,10000.0,19498.0,12.0,0.0,1.0,0.0
2,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,...,Руководитель высшего звена,Участие в основ. деятельности,360.0,от 20000 до 50000 руб.,30000.0,15470.0,3.0,15000.0,1.0,1.0
3,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,Новосибирская область,...,Специалист,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,13960.0,6.0,2500.0,1.0,0.0
4,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,Красноярский край,...,Специалист,Участие в основ. деятельности,12.0,от 10000 до 20000 руб.,15000.0,11890.0,6.0,8000.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,Чувашия,...,Специалист,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,8000.0,11750.0,4.0,1900.0,2.0,2.0
15996,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,Карелия,...,Специалист,Участие в основ. деятельности,24.0,от 20000 до 50000 руб.,12000.0,12350.0,6.0,1380.0,1.0,0.0
15997,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,Белгородская область,...,Специалист,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,9000.0,4915.0,10.0,2000.0,1.0,0.0
15998,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,Кабардино-Балкария,...,Специалист,Участие в основ. деятельности,36.0,от 10000 до 20000 руб.,12000.0,5860.0,3.0,2000.0,1.0,1.0


Видим, что все нетудоустроенные имеют соответствующие пропуски. Заполним для всех таких клиентов пропуски в полях GEN_INDUSTRY, GEN_TITLE, JOB_DIR значениями "Не трудоустроен", а в поле WORK_TIME - нулем, т.к. время работы в месяцах в данном случае нулевое.